# Training Deep Neural Networks

Training deep networks can entail a wide range of problems, from overfitting to vanishing/exploding gradients. This latter issue is especially troublesome as networks are trained by gradient descent, where the gradient is automatically computed through the backpropagation algorithm. If the gradient gets smaller and smaller when going backwards to the network then weights will not be updated and the network will never converge. Instead, in the opposite case, if the gradient gets bigget and bigger then weights updates will explode and the network will diverge.

In [37]:
import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

### Weights initialization
Keras defaults to **Glorot Initialization** with uniform distribution, where the boundaries of the distribution depend on the number of weights in the input and output layers. It is also possible to use the same initialization with a normal distribution. This type of initialization works best with linear layers and S-shaped activation functions like TanH, Sigmoid and Softmax.

In [5]:
Dense(10, kernel_initializer = 'glorot_normal')
Dense(10, kernel_initializer = 'glorot_uniform')

With RELU and RELU-derived activation functions it is often better to use **He Initialization**, which differs for the scale of the variance.

In [6]:
Dense(10, kernel_initializer = 'he_normal')
Dense(10, kernel_initializer = 'he_uniform')

Finally with SELU activation the preferred option is **LeCun initialization**.

In [7]:
Dense(10, kernel_initializer = 'lecun_normal')
Dense(10, kernel_initializer = 'lecun_uniform')

### Activation Functions
In the past the most common activation function was the S-shaped **sigmoid**. This can be a poor choice in many scenarios because of the vanishing gradient problem: when the sigmoid function saturates for values close to 0 or 1, the gradient is close to zero, making convergence very slow or even impossible. The **tanh** function also suffers from the same problem, but less because its non-saturation range is wider (from -1 to 1).

In [8]:
Dense(10, activation='sigmoid')
Dense(10, activation='tanh')

The **ReLU** function is the most common type of non-saturating activation function, both because it proved to work very well and because it is blazingly fast to compute. Its main drawback is that neurons using it could *die*: when the weighted sum of the inputs is negative for all instances in the training set, a ReLU adopting neuron will always output zero, making gradient descent unable to affect it. 

In [9]:
Dense(10, activation='relu')

Several improvements have been proposed to solve this issue. **Leaky ReLU** ensures that even for negative values, the function never becomes flat (and thus the gradient never becomes null). A very small slope ($\alpha$) is sufficient to ensure that neurons have a positive chance to wake-up. Additionally, the slope $\alpha$ could also be randomly drawn during training (RReLU) or be an additional parameter that should be learned during training (PReLU).

In [10]:
Dense(10)
keras.layers.LeakyReLU(alpha=0.1) # default alpha value is 0.3

A more interesting variant is the **Exponential Linear Unit (ELU)**. which substitutes the flat part on the left with an exponential function. Thus, ELU can take negative values and has a non-zero gradient everywhere. Furthermore, it is smooth, meaning that it doesn't have have the typical kink of ReLU functions. While ELU is slower to compute than ReLU, it often makes convergence faster because of its nice properties.

In [11]:
Dense(10, activation='elu')

But, the award of best activation function, when its application is feasible, goes to the **Scaled ELU (SELU)** function. A sequential network, composed solely of dense layers, with each hidden layer using the SELU activation function is guaranteed to **self-normalize** under certain conditions. This is a very nice property because in a normalized network the output of every layer will preserve a zero mean and a unitary standard deviation, solving the vanishing/exploding gradient problem.
* Inputs should be standardized (not min-max scaled!)
* Hidden layers should use LeCun initialization with normal distribution
* Network should be sequential
* All layers should be dense

In [12]:
Dense(10, activation='selu', kernel_initializer = 'lecun_normal')

### Batch Normalization
Batch normalization consits in adding an operation before or after the activation function of each hidden layer to shift and scale the inputs to make them approximately standardized. As the name suggests, the mean and standard deviation are evaluated over the current mini-batch, and not over the whole training sample. Notice that when a network has a BN layers as its very first layer, then it is not necessary to standardize the training data.
Since it could be difficult or even impossible to compute the mean and standard deviation at test time, as too few or even individual instances could be provided, most implementations of BN keep track of the training set mean and standard deviation through a moving average mechanism. 

In [18]:
Sequential([
    keras.layers.BatchNormalization(), # input
    Dense(10, activation='tanh'), # hidden
    keras.layers.BatchNormalization(), 
    Dense(1) # output
])

In this implementation the BN layers is after the activation function. In order for it to be placed before we must separate the activation function from its hidden layer.

In [19]:
Sequential([
    keras.layers.BatchNormalization(), # input
    Dense(10), # hidden
    keras.layers.BatchNormalization(), 
    keras.layers.Activation('tanh'),
    Dense(1) # output
])

### Optimizers
A simple, yet effective, way to improve over plain-vanilla stochastic gradient is through the idea of **momentum**. While regular SGD will take small steps down the slope, in the direction given by the gradient, momentum enchanched SGD will pick up speed at every step in the same direction, as a ball rolling down from a hill, until when terminal velocity is reached because of friction. In this way, the gradient is used only for determining the acceleration, not the speed (at least not directly). This trick makes convergence much faster when the slope is small and makes more difficult to remain stuck in local optima.
The momentum parameter determines the amount of friction in the environment (0 high friction, 1 no friction).

In [20]:
keras.optimizers.SGD(lr=.001, momentum=0.9)

Additionally, the **Nesterov Accelerated Gradient** method entails computing the gradient not at the current position $\theta$, but slightly ahead in the direction of the momentum $\theta + \beta m$. This will improve performances because generally the momentum vector points in the direction of the optimum. The main advantage is that this forward-looking behaviour reduces oscillations around the optimum, making convergence faster.

In [24]:
keras.optimizers.SGD(lr=.001, momentum=0.9, nesterov=True)

An issue with SGD is that descent is very quick in the direction of steepest descent, but this isn't necessarily the shortest direction to reach the global optimum (i.e. it doesn't necessarily point straight towards it). **AdaGrad** improves SGD by correcting the direction of descent so that it points more directly toward the global optimum. The result is achieved by using an adaptive learning rate, which decays at different rates across dimensions, with the goal of slowing down the descent over steeper dimensions. While the idea is nice, AdaGrad often stops too early when training neural networks.

In [23]:
keras.optimizers.adagrad()

Instead, **RMSprop** should be used, as it fixes the problem of AdaGrad by looking only at the gradients from the most recent iterations rather than accumulating all the gradients. To do so, it used exponential decay with the decay rate ($\rho$) typicall set to 0.9. 

In [27]:
keras.optimizers.RMSprop(lr=0.001, rho=0.9)

Finally, the **Adam** oprimizer (short version for adaptive momentum estimation) combines the idea of momentum estimation with RMSprop. The former, requires to keep track of an exponentially decaying average of past gradients, while the latter keeps track of an exponentially decaying average of past squared gradients. The momentum decay parameter ($\beta_1$) is often set to 0.9, while the learning rate scaling parameter ($\beta_2$) is usually set to 0.999.

Alernatively, **Nadam** is enhanced by using the nesterov trick, making convergence faster in many scenarios.

In [41]:
keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

### Learning rate scheduling
The learning rate is one of the most important hyperparameters, as it affects both the speed on convergence and the quality of the final result. Up to now we have always considered the learning rate to be kept constant over the training period, but this is generally not optimal as it would be better to have a large rate at the beginning, in order to make fast progress, and a small rate thereafter to ensure convergence to a good result.

A possibility is to use **power scheduling**: the learning rate is seet to be a function of the iteration number (t), like $\eta(t) = \eta_0/(1+t)^c$. The result is that it decreases over time at a decreasing rate (i.e. decay slows down).

In [29]:
keras.optimizers.Adam(lr=0.001, decay = 0.0001)

Instead, with **exponential scheduling** the learning rate will be reduced by a fixed ratio every some steps. E.g. $\eta(t) = \eta_0  0.1^{t/s}$

In [33]:
# decay from a starting value
def exp_decay(epoch):
    return 1 * 0.1**(epoch/20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)

# alternatively, decay from current lr
def exp_decay(epoch, lr):
    return lr * 0.1**(1/20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)

Another fairly simple, but not smooth, option is to use a **piecewise constant scheduling**. The main downside is that it requires to manually set the learning rate values and the training period when to use them.

In [34]:
def piecewise_constant_decay(epoch):
    if epoch < 5:
        return 0.01
    elif epoch <20:
        return 0.005
    else:
        return 0.001
    
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_decay)

### Regularization
Neural networks are very powerful because of their flexibility, but this also entails a high risk of overfitting the training set, especially when the amount of data is small compared to the size of the network. Similarly to Lasso and Ridge, we can apply **L1** and **L2 regularization** to constrain the weights of a neural network. The former is preferred to obtain a sparse model, while the latter yields a dense model.

In [35]:
Dense(10, kernel_regularizer = keras.regularizers.l1(0.01))
Dense(10, kernel_regularizer = keras.regularizers.l2(0.01))
Dense(10, kernel_regularizer = keras.regularizers.l1_l2(0.01, 0.01))

A very common regularization stategy is through the use of **dropout layers**. Its functioning is very simple: at every training iteration, every neuron (including the input neurons, excluding output ones) has a probability $p$ of being temporarily shut down. Dropped out neurons are ignored for the current training step, but could be active in the subsequent step. The parameter $p$ is called dropout rate and is generally set between 10% and 50%. The beneficial effect is partly due to the fact that the final network can be seen as an ensemble of smaller networks, which is not too dissimilar from how a random forest works. \
Then, after training, the whole network is used to make predictions without any neuron being dropped out. Connection weights are either scaled up during testing or scaled down during training to obtain a similar signal strenght during both phases. 

In [38]:
Sequential([
    Dropout(rate=0.2),
    Dense(100),
    Dropout(rate=0.2),
    Dense(100),
    Dropout(rate=0.2),
    Dense(1) # output
])